In [48]:
#!wget https://github.com/nvs543/AIML_B11_NLP/blob/c1666ca7a183eece7073c4c7e0946b8f46bfe1ed/bbc-fulltext.zip
#!wget https://github.com/nvs543/AIML_B11_NLP/blob/c7eb1d14b5e8d8ade64351006740b3e5763f5996/bbc.zip
#!wget /content/drive/MyDrive/Colab Notebooks/bbc.zip
!wget http://mlg.ucd.ie/files/datasets/bbc-fulltext.zip

--2022-11-27 06:42:38--  http://mlg.ucd.ie/files/datasets/bbc-fulltext.zip
Resolving mlg.ucd.ie (mlg.ucd.ie)... 137.43.93.132
Connecting to mlg.ucd.ie (mlg.ucd.ie)|137.43.93.132|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2874079 (2.7M) [application/zip]
Saving to: ‘bbc-fulltext.zip’

bbc-fulltext.zip    100%[===================>]   2.74M  --.-KB/s    in 0.1s    

2022-11-27 06:42:38 (20.3 MB/s) - ‘bbc-fulltext.zip’ saved [2874079/2874079]



In [49]:
!ls -lh


total 2.9M
-rw-r--r-- 1 root root 2.8M Mar 27  2022 bbc-fulltext.zip
-rw-r--r-- 1 root root 139K Nov 27 06:38 bbc.zip
drwx------ 6 root root 4.0K Nov 27 05:09 drive
drwxr-xr-x 1 root root 4.0K Nov 22 00:14 sample_data


In [ ]:
!unzip /content/bbc-fulltext.zip

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
import nltk
nltk.download('gutenberg')
nltk.download('punkt')
from urllib import request
import re
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
import os
import glob
import string

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
bbc_path = "/content/drive/MyDrive/Colab Notebooks/bbc/"
#bbc_path = "/content/bbc/"

In [28]:
def readFiles(path,clsId):
  fileNames = []
  fNames = []
  articles = []
  clsIds = []
  #print(glob.glob(path+"*.txt"))
  filenames=os.listdir(path)
  
  for ef in filenames:
    f = open(path+ef).readlines()
    articles.append(" ".join(f))
    clsIds.append(clsId)
    fNames.append(ef)
  
  return fNames,articles,clsIds
  

In [29]:
classNames = ["business","entertainment","politics","sport","tech"]
allArticles = []
allClsIds = []
allFileNames = []
for idx in range(len(classNames)):
  p = bbc_path+classNames[idx]+"/"
  fn,ad,cid = readFiles(p,idx)
  allArticles = allArticles + ad
  allClsIds = allClsIds + cid
  allFileNames = allFileNames + fn
 
print(len(allArticles),len(allClsIds),len(allFileNames))

2225 2225 2225


In [30]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

In [31]:
for i in range(len(allArticles)):
  allArticles[i] = allArticles[i].lower() #convert to lower case
  allArticles[i] = re.sub('<.*?>',' ',allArticles[i]) # remove html tags
  allArticles[i] = re.sub('[^a-zA-Z]',' ',allArticles[i]) #remove non-alphabet
  allArticles[i] = re.sub(r'\s+[a-zA-Z]\s+',' ',allArticles[i]) #remove single character
  allArticles[i] = re.sub(r'\s+', ' ',allArticles[i]) #remvoe multiple spaces
  allArticles[i] = REPLACE_BY_SPACE_RE.sub(' ',allArticles[i]) # replace REPLACE_BY_SPACE_RE symbols by space in text
  allArticles[i] = BAD_SYMBOLS_RE.sub('',allArticles[i]) #remove symbols that are in BAD_SYMBOLS from text
  allArticles[i] = allArticles[i].replace('x','') #remove x in text
  allArticles[i] = ' '.join(word for word in allArticles[i].split() if word not in STOPWORDS) # remove stop words

In [32]:
allArticles[0]

'us interest rate rise epected us interest rates epected rise fifth time since june following us federal reserve latest rate setting meeting later tuesday borrowing costs tipped rise quarter percentage point move comes recovery us economy world biggest shows signs robustness sustainability dollar record breaking decline meanwhile spooked markets along high oil prices raised concerns pace inflation seeing evidence inflation moving higher said ken kim analyst stone mccarthy research risk actually happening mr kim added borrowing costs could rise fed said move measured way combat price growth lift interest rates year lows prompted sluggish us global growth economic picture looking rosy fed implemented quarter percentage point rises june august september november although us economy grew annual rate three months september analysts warn fed careful move aggressively take wind recovery sails earlier month figures showed job creation still weak consumer confidence subdued think fed feels fair

In [33]:
#df with count vectorizer
cols = ['article', 'labels']
d = {'article':allArticles, 'labels':allClsIds}
df = pd.DataFrame(data=d, columns=cols)
df.head()

,article,labels
0,us interest rate rise epected us interest rate...,0
1,uk economy facing major risks uk manufacturing...,0
2,ask jeeves tips online ad revival ask jeeves b...,0
3,us company admits benin bribery us defence tel...,0
4,ticking budget facing us budget proposals laid...,0


In [34]:
vectorizer = CountVectorizer(stop_words = "english")
X =vectorizer.fit_transform(df['article'])
print(vectorizer.get_feature_names(), "\n")
print(vectorizer.vocabulary_, "\n")
print(len(vectorizer.get_feature_names()), "\n")
X

['aa', 'aaa', 'aaas', 'aac', 'aadc', 'aaliyah', 'aaltra', 'aamir', 'aan', 'aara', 'aarhus', 'aaron', 'aba', 'abacus', 'abandon', 'abandoned', 'abandoning', 'abandonment', 'abate', 'abatement', 'abating', 'abba', 'abbas', 'abbasi', 'abbey', 'abbot', 'abbott', 'abbreviated', 'abc', 'abd', 'abdellatif', 'abdication', 'abdomen', 'abdominal', 'abducted', 'abduction', 'abdul', 'abdullah', 'abdullatif', 'abebe', 'abensur', 'aberavon', 'aberdeen', 'aberration', 'aberystwyth', 'abetting', 'abeyance', 'abeyie', 'abhorrent', 'abi', 'abide', 'abided', 'abiding', 'abigail', 'abilities', 'ability', 'abish', 'abiyote', 'able', 'abn', 'abnormal', 'abnormality', 'abolish', 'abolished', 'abolishing', 'abolition', 'aborted', 'abortion', 'abortionist', 'abortions', 'abortive', 'aboutroughly', 'abraham', 'abramovich', 'abroad', 'abrupt', 'abruptly', 'absa', 'absence', 'absences', 'absent', 'absentee', 'absolute', 'absolutely', 'absolution', 'absorb', 'absorbed', 'absorbing', 'absorbs', 'abstain', 'abstenti

<2225x27500 sparse matrix of type '<class 'numpy.int64'>'
	with 307490 stored elements in Compressed Sparse Row format>

In [35]:
y = df['labels']


In [37]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(1780, 27500) (445, 27500) (1780,) (445,)


In [38]:
dt = DecisionTreeClassifier()
knn = KNeighborsClassifier(n_neighbors=3)
lc = SGDClassifier()
lr = LogisticRegression()
svc = SVC(kernel='linear')

models = [("DT",dt), ("KNN", knn), ("LC", lc), ("LR", lr), ("SVC", svc)]

In [39]:
from sklearn.preprocessing import StandardScaler
for mn, model in models:
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  print(mn,":",accuracy_score(y_test, y_pred))

DT : 0.8584269662921349
KNN : 0.7528089887640449
LC : 0.9707865168539326
LR : 0.9842696629213483
SVC : 0.9820224719101124


In [40]:
import joblib

In [42]:
folder_path = "/content/drive/MyDrive/Colab Notebooks/2A_BBC_Count_Vectorizer_Trained_Models/"
models_1 = [("DecisionTreeClassifier",dt), ("KNeighborsClassifier", knn), ("SGDClassifier", lc), ("LogisticRegression", lr), ("SVC", svc)]
for mn_1, model_1 in models_1:
  fname = folder_path+mn_1+".sav"
  joblib.dump(model_1, fname)

In [43]:

dt_pt = joblib.load(folder_path+"DecisionTreeClassifier.sav")
knn_pt = joblib.load(folder_path+"KNeighborsClassifier.sav")
lc_pt = joblib.load(folder_path+"SGDClassifier.sav")
lr_pt = joblib.load(folder_path+"LogisticRegression.sav")
svc_pt = joblib.load(folder_path+"SVC.sav")

models_pt = [("DT",dt_pt), ("KNN", knn_pt), ("LC", lc_pt), ("LR", lr_pt), ("SVC", svc_pt)]
for mn, model in models:
  acc = model.score(X_test, y_test)
  print(mn,":",acc)

DT : 0.8584269662921349
KNN : 0.7528089887640449
LC : 0.9707865168539326
LR : 0.9842696629213483
SVC : 0.9820224719101124
